# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!

In this lab, you'll practice your scraping skills on an online music magazine and events website called Resident Advisor.

## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://ra.co website. For reproducibility we will use the [Internet Archive](https://archive.org/) Wayback Machine to retrieve a version of this page from March 2019.

Start by navigating to the events page [here](https://web.archive.org/web/20210325230938/https://ra.co/events/us/newyork?week=2019-03-30) in your browser. It should look something like this:

<img src="images/ra_top.png">

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

## Write a Function to Scrape all of the Events on the Given Page

The function should return a Pandas DataFrame with columns for the `Event_Name`, `Venue`, and `Number_of_Attendees`.

Start by importing the relevant libraries, making a request to the relevant URL, and exploring the contents of the response with `BeautifulSoup`. Then fill in the `scrape_events` function with the relevant code.

In [47]:
# Relevant imports
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import numpy as np

In [48]:
EVENTS_PAGE_URL = "https://web.archive.org/web/20210326225933/https://ra.co/events/us/newyork?week=2019-03-30"

# Exploration: making the request and parsing the response
res = requests.get(EVENTS_PAGE_URL)
soup = BeautifulSoup(res.content, 'html.parser')

In [56]:
# Find the container with event listings in it
events_all = soup.find('div', {'data-tracking-id':'events-all'})
events_list = events_all.find('ul').find('li')
print(events_list.text[:500])

̸Sat, 30 MarUnterMania IIMary Yuzovskaya, Manni Dee, Umfang, Juana, The Lady MachineTBA - New YorkRARA Tickets457Cocoon New York: Sven Väth, Ilario Alicante, Butch & TaimurSven Vath, Butch, Taimur, Ilario Alicante99 Scott AveRARA Tickets407Horse Meat Disco - New York ResidencyHorse Meat Disco, The Carry Nation, Amber ValentineElsewhereRARA Tickets375RA PickRave: Underground Resistance All NightNomadico, Mark FlashNowadaysRARA Tickets232Detroit techno's most iconic crew dispatches two of its DJs 


In [50]:
# Find a list of events by date within that container
dates = events_list.findChildren(recursive=False)
print("0 index:", dates[0].text[:200])
print()
print("1 index:", dates[1].text[:200])
print()
print("2 index:", dates[2].text[:200])
print()

0 index: ̸Sat, 30 MarUnterMania IIMary Yuzovskaya, Manni Dee, Umfang, Juana, The Lady MachineTBA - New YorkRARA Tickets457Cocoon New York: Sven Väth, Ilario Alicante, Butch & TaimurSven Vath, Butch, Taimur, Il

1 index: 

2 index: ̸Sun, 31 MarSunday: Soul SummitNowadaysRARA Tickets132New Dad & Aaron Clark (Honcho)Aaron Clark, New DadAce Hotel3ParadiscoOccupy The DiscoLe Bain3Sunday Soiree: Unknown Showcase (Detroit)Ryan Dahl, H



In [63]:
# Extract the date (e.g. Sat, 30 Mar) from one of those containers
first_date = dates[0]
date = first_date.find('div', class_='sticky-header').text
date = date.strip("'̸")
date

'Sat, 30 Mar'

In [64]:
# Extract the name, venue, and number of attendees from one of the
# events within that container
first_date_events = first_date.findChildren('ul')
first_event = first_date_events[0]
name = first_event.find('h3').text
print("Name:", name)
venue_and_attendees = first_event.find_all('div', attrs = {"height": 30})
venue = venue_and_attendees[0].text
print("Venue:", venue)
print('Date:', date)
attendees = int(venue_and_attendees[-1].text)
print("Number of attendees:", attendees)

Name: UnterMania II
Venue: TBA - New York
Date: Sat, 30 Mar
Number of attendees: 457


In [66]:
last_event = first_date_events[-1]
name = last_event.find('h3').text
print("Name:", name)
venue_and_attendees = last_event.find_all('div', attrs = {"height": 30})
venue = venue_and_attendees[-1].text
print("Venue:", venue)
print('Date:', date)
attendees = int(venue_and_attendees[-1].text)
print("Number of attendees:", attendees)

Name: Petra, Matthusen & Lang, White & Pitsiokos, and Zorn
Venue: H0L0
Date: Sat, 30 Mar


ValueError: invalid literal for int() with base 10: 'H0L0'

In [67]:
try:
    attendees = int(venue_and_attendees[-1].text)
except ValueError:
    attendees = np.nan
print('Number of attendees:', attendees)

Number of attendees: nan


In [69]:
# Loop over all of the event entries, extract this information
# from each, and assemble a dataframe
rows = []
for date_container in dates:
    if not date_container.text:
        continue
    date = date_container.find("div", class_="sticky-header").text
    date = date.strip("'̸")
    events = date_container.findChildren("ul")
    for event in events:
        name = event.find("h3").text
        venue_and_attendees = event.findAll("div", attrs={"height": 30})
        venue = venue_and_attendees[0].text
        try:
            num_attendees = int(venue_and_attendees[-1].text)
        except ValueError:
            num_attendees = np.nan
        rows.append([name, venue, date, num_attendees])
df = pd.DataFrame(rows)
df

,0,1,2,3
0,UnterMania II,TBA - New York,"Sat, 30 Mar",457.0
1,"Cocoon New York: Sven Väth, Ilario Alicante, B...",99 Scott Ave,"Sat, 30 Mar",407.0
2,Horse Meat Disco - New York Residency,Elsewhere,"Sat, 30 Mar",375.0
3,Rave: Underground Resistance All Night,Nowadays,"Sat, 30 Mar",232.0
4,"Believe You Me // Beta Librae, Stephan Kimbel,...",TBA - New York,"Sat, 30 Mar",89.0
...,...,...,...,...
114,A Night at the Baths,C'mon Everybody,"Fri, 5 Apr",1.0
115,Blaqk Audio,Music Hall of Williamsburg,"Fri, 5 Apr",1.0
116,Erik the Lover,Erv's,"Fri, 5 Apr",1.0
117,Wax On Vissions,Starliner,"Fri, 5 Apr",1.0


In [70]:
# Bring it all together in a function that makes the request, gets the
# list of entries from the response, loops over that list to extract the
# name, venue, date, and number of attendees for each event, and returns
# that list of events as a dataframe

def scrape_events(events_page_url):
    #Your code here
    res = requests.get(EVENTS_PAGE_URL)
    soup = BeautifulSoup(res.content, 'html.parser')
    events_all = soup.find('div', {'data-tracking-id':'events-all'})
    events_list = events_all.find('ul').find('li')
    dates = events_list.findChildren(recursive=False)
    rows = []
    for date_container in dates:
        if not date_container.text:
            continue
        date = date_container.find("div", class_="sticky-header").text
        date = date.strip("'̸")
        events = date_container.findChildren("ul")
        for event in events:
            name = event.find("h3").text
            venue_and_attendees = event.findAll("div", attrs={"height": 30})
            venue = venue_and_attendees[0].text
            try:
                num_attendees = int(venue_and_attendees[-1].text)
            except ValueError:
                num_attendees = np.nan
            rows.append([name, venue, date, num_attendees])
    df = pd.DataFrame(rows)
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

In [71]:
# Test out your function
scrape_events(EVENTS_PAGE_URL)

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,UnterMania II,TBA - New York,"Sat, 30 Mar",457.0
1,"Cocoon New York: Sven Väth, Ilario Alicante, B...",99 Scott Ave,"Sat, 30 Mar",407.0
2,Horse Meat Disco - New York Residency,Elsewhere,"Sat, 30 Mar",375.0
3,Rave: Underground Resistance All Night,Nowadays,"Sat, 30 Mar",232.0
4,"Believe You Me // Beta Librae, Stephan Kimbel,...",TBA - New York,"Sat, 30 Mar",89.0
...,...,...,...,...
114,A Night at the Baths,C'mon Everybody,"Fri, 5 Apr",1.0
115,Blaqk Audio,Music Hall of Williamsburg,"Fri, 5 Apr",1.0
116,Erik the Lover,Erv's,"Fri, 5 Apr",1.0
117,Wax On Vissions,Starliner,"Fri, 5 Apr",1.0


## Write a Function to Retrieve the URL for the Next Page

As you scroll down, there should be a button labeled "Next Week" that will take you to the next page of events. Write code to find that button and extract the URL from it.

This is a relative path, so make sure you add `https://web.archive.org` to the front to get the URL.

![next page](images/ra_next.png)

In [72]:
# Find the button, find the relative path, create the URL for the current `soup`
svg = soup.find("svg", attrs={"aria-label": "Right arrow"})
svg_parent = svg.parent
link = svg.parent.previousSibling
relative_path = link.get("href")
next_page_url = "https://web.archive.org" + relative_path
next_page_url

'https://web.archive.org/web/20210326225933/https://ra.co/events/us/newyork?week=2019-04-06'

In [73]:
# Fill in this function, to take in the current page's URL and return the
# next page's URL
def next_page(url):
    #Your code here
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    svg = soup.find("svg", attrs={"aria-label": "Right arrow"})
    svg_parent = svg.parent
    link = svg.parent.previousSibling
    relative_path = link.get("href")
    next_page_url = "https://web.archive.org" + relative_path
    return next_page_url

In [74]:
# Test out your function
next_page(EVENTS_PAGE_URL)

'https://web.archive.org/web/20210326225933/https://ra.co/events/us/newyork?week=2019-04-06'

## Scrape the Next 500 Events

In other words, repeatedly call `scrape_events` and `next_page` until you have assembled a dataframe with at least 500 rows.

Display the data sorted by the number of attendees, greatest to least.

We recommend adding a brief `time.sleep` call between `requests.get` calls to avoid rate limiting.

In [75]:
# Your code here
overall_df = pd.DataFrame()
current_url = EVENTS_PAGE_URL
while overall_df.shape[0] <= 500:
    df = scrape_events(current_url)
    time.sleep(.2)
    overall_df = pd.concat([overall_df, df])
    current_url = next_page(current_url)
    time.sleep(.2)
overall_df

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,UnterMania II,TBA - New York,"Sat, 30 Mar",457.0
1,"Cocoon New York: Sven Väth, Ilario Alicante, B...",99 Scott Ave,"Sat, 30 Mar",407.0
2,Horse Meat Disco - New York Residency,Elsewhere,"Sat, 30 Mar",375.0
3,Rave: Underground Resistance All Night,Nowadays,"Sat, 30 Mar",232.0
4,"Believe You Me // Beta Librae, Stephan Kimbel,...",TBA - New York,"Sat, 30 Mar",89.0
...,...,...,...,...
114,A Night at the Baths,C'mon Everybody,"Fri, 5 Apr",1.0
115,Blaqk Audio,Music Hall of Williamsburg,"Fri, 5 Apr",1.0
116,Erik the Lover,Erv's,"Fri, 5 Apr",1.0
117,Wax On Vissions,Starliner,"Fri, 5 Apr",1.0


## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!